<html>
   <head>
      <title>Cleaning and Engineering</title>
   </head>
   <body>
      <h1><font size="6">Part 1: Cleaning and Engineering</font></h1>
       <p><font size="3">This notebook consists of data cleaning and exploration. Also first steps in feature engineering take place.<br> It is structured in the following way:</font></p>
      <ul style="list-style-type:none;line-height: 2;">
         <li><font size="3"><a href='#1'>1. Loading the Data</a></font></li>
         <li><font size="3"><a href='#2'>2. First Look at the Data</a></font></li>
         <ul style="margin-left: 1em; padding-left: 1em;list-style-type:none">
         <li><font size="3"><a href='#2.1'>2.1 Feature Dictionary</a></font></li>
         </ul>
         <li><font size="3"><a href='#3'>3. Data Cleaning</a></font></li>
         <ul style="margin-left: 1em; padding-left: 1em;list-style-type:none">
         <li><font size="3"><a href='#3.1'>3.1 Correcting Visit ID</a></font></li>
         <li><font size="3"><a href='#3.2'>3.2 Correcting Product Group and Line</a></font></li>
         <li><font size="3"><a href='#3.3'>3.3 NaNs</a></font></li>
         <li><font size="3"><a href='#3.4'>3.4 Data Types</a></font></li>
         <li><font size="3"><a href='#3.5'>3.5 Duplicates</a></font></li>         
         </ul>          
         <li><font size="3"><a href='#4'>4. Understanding particular Features in Detail</a></font></li>
         <li><font size="3"><a href='#5'>5. Feature Engineering</a></font></li>
         <ul style="margin-left: 1em; padding-left: 1em;list-style-type:none">
         <li><font size="3"><a href='#3.1'>5.1 Cities by Code</a></font></li>
         <li><font size="3"><a href='#3.2'>5.2 Timespan</a></font></li>
         <li><font size="3"><a href='#3.3'>5.3 Weekdays</a></font></li>
         <li><font size="3"><a href='#3.4'>5.4 Article Genres</a></font></li>
         <li><font size="3"><a href='#3.5'>5.5 Total price per article purchase</a></font></li>    
         <li><font size="3"><a href='#3.2'>5.6 Value and number of items of shopping carts</a></font></li>
         <li><font size="3"><a href='#3.3'>5.7 Stockprice per day</a></font></li>
         <li><font size="3"><a href='#3.4'>5.8 First date of purchase and purchase type</a></font></li>
         <li><font size="3"><a href='#3.5'>5.9 Final Feature Dictionary</a></font></li>               
         </ul>            
      </ul>
   </body>
</html>

## Importing libraries

In [1]:
# Importing the necessary libraries
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import glob
import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, f1_score, precision_recall_fscore_support
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score, cross_val_predict

## Setting options

In [2]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 400)

In [3]:
#adjusting notebook width
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:90% !important; }</style>"))

In [4]:
#markdowntables layout

In [5]:
%%html
<style>
table {float:left}
table td, table th, table tr {text-align:left !important;}
</style>

<a id='1'></a>

# 1. Loading the data

In [6]:
#sales dataframe
df_base = pd.read_csv("./Data/TTdata.csv")

//anaconda3/envs/test/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (5,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
#copy for working
df = df_base.copy()

In [8]:
#dataframe with citynames
#https://developers.google.com/adwords/api/docs/appendix/geotargeting
df_geotargets = pd.read_csv("./Data/geotargets.csv") 

In [9]:
#Another project of the course looked at stock prices. Taking them into the data just for fun (won't have any further use)
df_stock = pd.read_csv("./Data/daily_TTI.DE.csv")

<a id='2'></a>

# 2. First Look at the Data

In [10]:
df.head()

,visit_id_n,unser_id_n,visit_number,date,article_name,article_id,ga_category,brand,price,quantity,device_category,device_browser,device_os,country,region,city_id,product_line,product_group,product_line_r,product_group_r,campaign_involved
0,551960,553811,1.0,20180910.0,Geschenk Gutschein 10 EUR,200000000010,Herren,(not set),10.0,1.0,desktop,Firefox,Windows,Germany,Lower Saxony,1004478,Herren,Herren,Herren,Herren,False
1,242735,254033,1.0,20180104.0,Geschenkgutschein 10 EUR,200000000010,Damen/Geschenkgutscheine,(not set),10.0,1.0,desktop,Firefox,Windows,Germany,Lower Saxony,9048892,Herren,Herren,Herren,Herren,True
2,680200,620544,2.0,20180612.0,Geschenk Gutschein 10 EUR,200000000010,Damen/Geschenkgutscheine,(not set),10.0,1.0,desktop,Chrome,Windows,Germany,Berlin,1003854,Herren,Herren,Herren,Herren,True
3,44920,609593,46.0,20180913.0,Geschenk Gutschein 10 EUR,200000000010,Herren,(not set),10.0,1.0,mobile,Safari,iOS,Germany,Lower Saxony,1004564,Herren,Herren,Herren,Herren,True
4,704159,73553,2.0,20180729.0,Geschenk Gutschein 10 EUR,200000000010,Herren,(not set),10.0,1.0,desktop,Chrome,Windows,Germany,Bavaria,1004246,Herren,Herren,Herren,Herren,True


In [11]:
df.tail()

,visit_id_n,unser_id_n,visit_number,date,article_name,article_id,ga_category,brand,price,quantity,device_category,device_browser,device_os,country,region,city_id,product_line,product_group,product_line_r,product_group_r,campaign_involved
3016738,33526,316730,2.0,20181122.0,Unifarbener Ösenschal,4260478143821,(not set),TT_Casual,36.97,1.0,mobile,Chrome,Android,Germany,Berlin,1003854,Home & Living,Vorhänge,Home & Living,Schuhe & Accessoires,True
3016739,520195,51192,16.0,20181201.0,Unifarbener Ösenschal,4260478143821,Home & Living/Vorhänge,TT_Casual,42.85,2.0,mobile,Safari,iOS,Germany,North Rhine-Westphalia,1004707,Home & Living,Vorhänge,Home & Living,Schuhe & Accessoires,True
3016740,18132,393215,2.0,20181124.0,Unifarbener Ösenschal,4260478143821,(not set),TT_Casual,37.39,2.0,mobile,Safari,iOS,Germany,Berlin,1003854,Home & Living,Vorhänge,Home & Living,Schuhe & Accessoires,True
3016741,7155,211171,NaN,NaN,NaN,anonymized,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Shirts unter 20 Euro,Shirts unter 20 Euro,Shirts unter 20 €,T-Shirts & Langarmshirts,NaN
3016742,7155,211171,NaN,NaN,NaN,cliqz.com/tracking,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Damen,langarm,Damen,T-Shirts & Langarmshirts,NaN


In [12]:
df.shape

(3016743, 21)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3016743 entries, 0 to 3016742
Data columns (total 21 columns):
visit_id_n           int64
unser_id_n           int64
visit_number         float64
date                 float64
article_name         object
article_id           object
ga_category          object
brand                object
price                float64
quantity             float64
device_category      object
device_browser       object
device_os            object
country              object
region               object
city_id              object
product_line         object
product_group        object
product_line_r       object
product_group_r      object
campaign_involved    object
dtypes: float64(4), int64(2), object(15)
memory usage: 483.3+ MB


In [14]:
df.describe()

,visit_id_n,unser_id_n,visit_number,date,price,quantity
count,3.016743e+06,3.016743e+06,3.016732e+06,3.016732e+06,3.016732e+06,3.016732e+06
mean,4.102715e+05,3.108908e+05,1.058004e+01,2.018055e+07,2.668266e+01,1.018650e+00
std,2.366932e+05,1.795393e+05,2.962561e+01,4.429808e+03,1.790388e+01,1.586700e-01
min,1.000000e+00,1.000000e+00,1.000000e+00,2.017110e+07,0.000000e+00,1.000000e+00
25%,2.053810e+05,1.551850e+05,1.000000e+00,2.018032e+07,1.445000e+01,1.000000e+00
50%,4.107240e+05,3.107390e+05,3.000000e+00,2.018071e+07,2.420000e+01,1.000000e+00
75%,6.152210e+05,4.659450e+05,9.000000e+00,2.018103e+07,3.361000e+01,1.000000e+00
max,8.201950e+05,6.226180e+05,1.616000e+03,2.019023e+07,2.512600e+02,1.000000e+01


In [15]:
df.sample(n=15)

,visit_id_n,unser_id_n,visit_number,date,article_name,article_id,ga_category,brand,price,quantity,device_category,device_browser,device_os,country,region,city_id,product_line,product_group,product_line_r,product_group_r,campaign_involved
2377175,342011,76438,7.0,20190129.0,Gestreifte Bluse,4061945276765,Damen/Damenmode/Blusen/Langarm,TT_Denim,42.01,1.0,mobile,Samsung Internet,Android,Germany,Saxony,1004905,Damen,Langarm,Damen,T-Shirts & Langarmshirts,True
2450067,428632,522341,1.0,20180819.0,T-Shirt mit Underlayer in Melange-Optik,4062105088105,Herren/Herrenmode/T-Shirts & Langarmshirts/T-S...,TT_Casual,14.70,1.0,desktop,Edge,Windows,Germany,Bavaria,9048278,Herren,T-Shirts,Herren,T-Shirts & Langarmshirts,True
692434,552572,98639,1.0,20180109.0,Aedan Slim,4059953320952,Herren/Herrenmode/Jeans/Slim,TT_Denim,21.00,1.0,tablet,Chrome,Android,Germany,Lower Saxony,1004478,Herren,Slim,Herren,Jeans,True
704153,67709,54668,1.0,20171217.0,gestreiftes Shirt mit Schleife,4059953351543,Damen/Damenmode/T-Shirt & Langarm-Shirts/Langa...,TT_Denim,12.76,1.0,tablet,Chrome,iOS,Germany,Baden-Wurttemberg,1004026,Damen,Langarmshirts,Damen,T-Shirts & Langarmshirts,True
1686308,811720,593103,4.0,20180430.0,Naomi Campbell: Kleid mit Taillengürtel,4060586780136,Damen/Damenmode/Kleider & Röcke,(not set),58.82,1.0,mobile,Safari,iOS,Germany,Saxony,1004948,Kurze Kleider,Kurze Kleider,Damen,Kleider,True
2757292,220578,583832,1.0,20181211.0,Kariertes Hemd,4062105552231,Herren/Herrenmode/Hemden/Langarm,TT_Casual,20.16,1.0,desktop,Firefox,Windows,Germany,Bavaria,1004320,Herren,Langarm,Herren,T-Shirts & Langarmshirts,False
543060,398952,143763,7.0,20171202.0,gesteppter Mantel mit Kapuze,4059491958785,"Damen/Damenmode/Jacken, Mäntel & Westen/Mäntel...",TT_Casual,131.69,1.0,desktop,Chrome,Windows,Germany,Saxony-Anhalt,1004960,Damen,Mäntel & Parkas,Damen,"Jacken, Mäntel & Westen",True
64232,55769,254292,8.0,20190105.0,klassischer Leder-Gürtel,4043825511951,Damen/Damenschuhe & Accessoires/Accessoires/Gü...,TT_Casual,19.32,1.0,mobile,Chrome,Android,Germany,Hesse,1004363,Damen,Gürtel,Damen,Schuhe & Accessoires,True
1701498,113562,541163,4.0,20180825.0,Carrie Straight Jeans,4060586809004,Damen/Damenmode/Jeans/Straight,TT_Casual,37.80,1.0,mobile,Chrome,Android,Germany,Berlin,1003854,Damen,Straight,Damen,Jeans,True
558829,478766,43179,4.0,20171111.0,Strickshirt im Tunika-Stil,4059491980496,Damen/Damenmode/Pullover & Sweatshirts/Pullover,TT_Casual,33.65,1.0,tablet,Chrome,iOS,Germany,Thuringia,1004974,Damen,Pullover,Damen,Pullover & Sweatshirts,False


<a id='2.1'></a>

## Feature dictionary ( my impressions )

| Feature No. | Feature Name | Description |
| :-: | :-: | :-: |
| 1 | visit_id_n | Unique ID of a single shopping cart |
| 2 | unser_id_n | Unique ID of a user (misstyped obviously) |
| 3 | visit_number | Not entirely clear; either number of visits to the shop or number of total purchases |
| 4 | date | date the purchase was made |
| 5 | article_name | Name of the article that was bought |
| 6 | article_id | ID of the article that was bought |
| 7 | ga_category | Category the article falls in ( partly the path in the webshop to the article ) |
| 8 | brand | Brand of the article |
| 9 | price | Price of the article |
| 10 | quantity | Number of the article that were bought |
| 11 | device_category | Category the user device falls in |
| 12 | device_browser | Category the user browser falls in |
| 13 | device_os | Category the user OS falls in |
| 14 | country | country location of the user |
| 15 | region | region location of the user |
| 16 | city_id | geoID of the town location of the user |
| 17 | product_line | Product line of the article |
| 18 | product_group | Product group of the article |
| 19 | product_line_r | not clear what the r stands for; might stand for retail |
| 20 | product_group_r | not clear what the r stands for; might stand for retail |
| 21 | campaign_involved | not clear what the campaign stands for |

<a id='3'></a>

# 3. Data cleaning

In [16]:
#rename columns
df.rename(columns={'visit_id_n':'visit_id',
                          'unser_id_n':'user_id',
                          'ga_category':'category'}, inplace=True)

<a id='3.1'></a>

## 3.1 Correcting Visit ID
After extensive research in a later part of the project it came up, that some visit id's were assigned to multiple users. Probably because the german and austrian webshop data were just put together and not filtered. As such define visit id as the sum of visit id and user id to get a unique number.

In [17]:
df["visit_id"] = df["visit_id"].apply(lambda x: str(x)) + df["user_id"].apply(lambda x: str(x))
df = df.astype({"visit_id": "int64"})

<a id='3.2'></a>

## 3.2 Correcting Product group and line

At a talk with the people responsible for the data they concluded that product_line_r and product_group_r are the correct features, hence the ones without _r will be dropped.

In [18]:
df = df.drop(columns=["product_line", "product_group"])
df.rename(columns={"product_line_r": "product_line", "product_group_r": "product_group"}, inplace=True)

<a id='3.3'></a>

## 3.3 NaNs

In [19]:
df.isna().sum()

visit_id              0
user_id               0
visit_number         11
date                 11
article_name         11
article_id            0
category             11
brand                11
price                11
quantity             11
device_category      11
device_browser       11
device_os            11
country              11
region               11
city_id              11
product_line          0
product_group         0
campaign_involved    11
dtype: int64

In [20]:
df.dropna(inplace=True) #just 11 , so drop them

In [21]:
df.isna().sum()

visit_id             0
user_id              0
visit_number         0
date                 0
article_name         0
article_id           0
category             0
brand                0
price                0
quantity             0
device_category      0
device_browser       0
device_os            0
country              0
region               0
city_id              0
product_line         0
product_group        0
campaign_involved    0
dtype: int64

In [22]:
#number of (not set)
df.isin(['(not set)']).sum(axis=0)

visit_id                  0
user_id                   0
visit_number              0
date                      0
article_name              0
article_id                0
category              41935
brand                398925
price                     0
quantity                  0
device_category           0
device_browser            0
device_os                75
country                1178
region                 7731
city_id               93307
product_line          31227
product_group         31227
campaign_involved         0
dtype: int64

For now I will keep the (not set)'s as a feature.

<a id='3.4'></a>

## 3.4 Datatypes

In [23]:
#change visit_number and quantity to int as float does not make sense
df = df.astype({'visit_number': 'int64', "quantity": "int64", "article_id": "int64"})
#device stuff and brand to category
df = df.astype({'device_category': 'category', "device_browser": "category", "device_os": "category", "brand": "category"})
#campaign_involved to bool
df.campaign_involved = df.campaign_involved.astype("bool")

In [25]:
#date to date format
df["date"]= pd.to_datetime(df["date"],format='%Y%m%d')

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3016732 entries, 0 to 3016740
Data columns (total 19 columns):
visit_id             int64
user_id              int64
visit_number         int64
date                 datetime64[ns]
article_name         object
article_id           int64
category             object
brand                category
price                float64
quantity             int64
device_category      category
device_browser       category
device_os            category
country              object
region               object
city_id              object
product_line         object
product_group        object
campaign_involved    bool
dtypes: bool(1), category(4), datetime64[ns](1), float64(1), int64(5), object(7)
memory usage: 359.6+ MB


### Price seems wrong for the following, as the original price is around 5-8, so discounts are in the price already

In [27]:
df.iloc[658646]

visit_id                                                  424670120783
user_id                                                         120783
visit_number                                                         4
date                                               2018-07-29 00:00:00
article_name                                  Bandana in Melange-Optik
article_id                                               4059953239858
category             Kinder/Kindermode/Baby Mädchen 0 - 17 Monate/A...
brand                                                        TT_Casual
price                                                             0.38
quantity                                                             1
device_category                                                desktop
device_browser                                                 Firefox
device_os                                                      Windows
country                                                        Germany
region

### Renaming a particular article

In [28]:
d = {'Geschenk Gutschein 10 EUR':'Geschenkgutschein 10 EUR','Geschenk Gutschein 20 EUR':'Geschenkgutschein 20 EUR','Geschenk Gutschein 50 EUR':'Geschenkgutschein 50 EUR'}
df = df.replace(d)

<a id='3.5'></a>

## 3.5 Duplicates

### Duplicated rows

In [29]:
df.duplicated().sum()

632

In [30]:
duplicateRowsDF = df[df.duplicated(keep=False)]

In [31]:
duplicateRowsDF.head()

,visit_id,user_id,visit_number,date,article_name,article_id,category,brand,price,quantity,device_category,device_browser,device_os,country,region,city_id,product_line,product_group,campaign_involved
118,22892413921,413921,45,2018-07-11,Geschenkgutschein 20 EUR,200000000020,Herren,(not set),20.00,1,desktop,Chrome,Windows,Austria,Upper Austria,1000827,Herren,Herren,False
119,22892413921,413921,45,2018-07-11,Geschenkgutschein 20 EUR,200000000020,Herren,(not set),20.00,1,desktop,Chrome,Windows,Austria,Upper Austria,1000827,Herren,Herren,False
675,177945413228,413228,1,2019-01-01,Sneaker-Socken im Doppelpack,4002702123007,"Kinder/Kindermode/Mädchen 1,5 - 7 Jahre/Schuh...",TT_Casual,2.31,1,mobile,Chrome,Android,Germany,Berlin,1003854,Kinder,Schuhe & Accessoires,True
676,177945413228,413228,1,2019-01-01,Sneaker-Socken im Doppelpack,4002702123007,"Kinder/Kindermode/Mädchen 1,5 - 7 Jahre/Schuh...",TT_Casual,2.31,1,mobile,Chrome,Android,Germany,Berlin,1003854,Kinder,Schuhe & Accessoires,True
1010,601457557264,557264,1,2018-12-25,Sneaker-Socken im Doppelpack,4002702123014,"Kinder/Kindermode/Mädchen 1,5 - 7 Jahre/Schuh...",TT_Casual,3.18,10,mobile,Chrome,iOS,Germany,Baden-Wurttemberg,1004022,Kinder,Schuhe & Accessoires,False


Note that it is not sure whether these rows are truly duplicates or just multiple purchases where the quantity wasn't set properly or the articles have the same name and price and for example different colors which this data does not tell us.

In [32]:
#df.drop_duplicates(keep="first", inplace=True)
#df.reset_index()

checking number of visits to number of entries (45 for this user)

In [33]:
df.user_id[413921]

230677

In [34]:
df[df['user_id'] == 413921].shape

(27, 19)

### Duplicates in visit_id

In [35]:
df.visit_id.max()

820195135969

In [36]:
df.visit_id.duplicated().sum()

2181687

In [37]:
df_visits = df[df.duplicated(subset="visit_id") == True]

In [38]:
df_uniquevisits = df.drop_duplicates(subset="visit_id")

In [39]:
df_visits.shape

(2181687, 19)

In [40]:
df_uniquevisits.shape

(835045, 19)

In [41]:
df_visits.sort_values(by=['visit_id']).head()

,visit_id,user_id,visit_number,date,article_name,article_id,category,brand,price,quantity,device_category,device_browser,device_os,country,region,city_id,product_line,product_group,campaign_involved
2972494,234440,40,1,2018-11-18,Schlichter Snood,4062105772851,Hoodies/Herren,TT_Denim,17.99,1,mobile,Chrome,Android,Austria,Lower Austria,1000750,Herren,Pullover & Sweatshirts,True
2817928,234440,40,1,2018-11-18,Hoodie mit Print,4062105670652,Hoodies/Herren,TT_Casual,15.00,1,mobile,Chrome,Android,Austria,Lower Austria,1000750,Herren,Herren,True
2538377,234440,40,1,2018-11-18,Basic Herren-Strick-Pullover,4062105188904,Herren,TT_Casual,14.99,1,mobile,Chrome,Android,Austria,Lower Austria,1000750,Herren,Pullover & Sweatshirts,True
2375327,276103,76103,22,2019-02-21,Culver Skinny Jeans,4061945274082,Herren/Herrenmode/Jeans/Skinny,TT_Denim,42.01,1,mobile,Safari,iOS,Germany,Berlin,1003854,Herren,Jeans,True
2647054,290200,200,43,2018-11-03,Steppjacke,4062105383903,Steppjacken,TT_Casual,31.51,1,mobile,Chrome,Android,Germany,Brandenburg,1003866,Herbst Outfits,Herbst Outfits,True


In [42]:
df_uniquevisits.sort_values(by=["visit_id"]).head()

,visit_id,user_id,visit_number,date,article_name,article_id,category,brand,price,quantity,device_category,device_browser,device_os,country,region,city_id,product_line,product_group,campaign_involved
320852,234440,40,1,2018-11-18,Basic Herren-Strick-Pullover,4059491520210,Herren,TT_Casual,14.99,1,mobile,Chrome,Android,Austria,Lower Austria,1000750,Herren,Pullover & Sweatshirts,True
2079241,276103,76103,22,2019-02-21,Henley T-Shirt,4060868345305,Herren/Herrenmode/T-Shirts & Langarmshirts/T-S...,TT_Casual,10.91,1,mobile,Safari,iOS,Germany,Berlin,1003854,Herren,T-Shirts & Langarmshirts,True
2645748,290200,200,43,2018-11-03,Steppjacke,4062105383569,Steppjacken/Damen,TT_Casual,31.50,1,mobile,Chrome,Android,Germany,Brandenburg,1003866,Damen,"Jacken, Mäntel & Westen",True
2830805,336120,20,2,2018-10-25,Nela Extra Skinny Jeans,4062105674247,Damen/Damenmode/Jeans/Skinny,TT_Denim,33.61,1,mobile,Chrome,Android,Germany,Saarland,9048444,Damen,Jeans,True
321178,366022,6022,8,2017-12-30,Basic Herren-Strick-Pullover,4059491520241,Feeling Blue Men,TT_Casual,16.80,1,desktop,Safari,Macintosh,Germany,Berlin,1003854,Feeling Blue Men,Feeling Blue Men,True


Duplicates in the visit_id surely denote multiple articles that were bought at the same visit, thus dropping any will not make sense. Also there were 820194 unique visit_id's, hence we know that the data consists of this number of shopping cart purchases.

<a id='4'></a>

# 4. Understanding particular Features in Detail

## Article names and id

In [43]:
df.article_name.value_counts().head(10)

Alexa Slim Jeans                          62458
Josh Regular Slim Jeans                   41092
T-Shirt mit Schrift-Print                 38895
T-Shirt mit Print                         29257
T-Shirt mit Logo-Print                    28231
Marvin Straight Jeans                     27405
Alexa Straight Jeans                      22816
Damen-Basic-Pullover mit V-Ausschnitt     19512
Jim Slim Bermuda Shorts                   18002
Basic Herren-Pullover mit V-Ausschnitt    17812
Name: article_name, dtype: int64

In [44]:
df.article_id.value_counts().head(10)

4059066698719    1728
4059066698511    1284
4002702235090    1215
4059066698726    1176
4059066698733    1054
4059953383155     998
4059953383148     908
4059953383162     887
4059953383179     861
4059491897060     819
Name: article_id, dtype: int64

## Unique devices 
lets check the entries and number of occurance of the elements of the device features

In [45]:
df.device_browser.value_counts()

Chrome                      994731
Safari                      793300
Firefox                     589700
Samsung Internet            256273
Edge                        155931
Internet Explorer            88370
Android Webview              63918
Opera                        29834
Safari (in-app)              28517
Amazon Silk                  14662
YaBrowser                      354
SeaMonkey                      337
Puffin                         235
Maxthon                        217
Android Browser                 94
Mozilla Compatible Agent        85
Iron                            58
UC Browser                      43
Playstation 4                   26
BlackBerry                      14
MRCHROME                        12
Seznam                          12
Coc Coc                          4
Opera 9.4                        3
Nintendo Browser                 2
Name: device_browser, dtype: int64

In [46]:
df.device_category.value_counts()

desktop    1371610
mobile     1091610
tablet      553512
Name: device_category, dtype: int64

In [47]:
df.device_os.value_counts()

Windows          1204420
Android           923303
iOS               718614
Macintosh         156569
Linux              11633
Chrome OS           1778
BlackBerry           222
(not set)             75
Windows Phone         70
Playstation 4         26
Tizen                 19
Nintendo WiiU          2
FreeBSD                1
Name: device_os, dtype: int64

Nothing too surprising so far. Later we can check if there were differences in the purchases between the different features.

## Campaign_involved:

In [48]:
camp_true = df[df.campaign_involved == True]

In [49]:
len(camp_true)

2278279

In [50]:
camp_true.sample(n=5)

,visit_id,user_id,visit_number,date,article_name,article_id,category,brand,price,quantity,device_category,device_browser,device_os,country,region,city_id,product_line,product_group,campaign_involved
1146695,106881619270,619270,35,2018-07-25,gemusterte Bluse,4060586141302,Damen/Damenmode/Blusen/langarm,TT_Denim,22.68,1,desktop,Firefox,Windows,Germany,Mecklenburg-Vorpommern,(not set),Damen,T-Shirts & Langarmshirts,True
2326132,482378130707,130707,20,2018-12-29,Langarmshirt mit doppeltem Kragen,4061945069558,Damen/Damenmode/T-Shirts & Langarmshirts/Langa...,TT_Casual,29.61,1,mobile,Chrome,Android,Germany,Lower Saxony,1004538,Damen,T-Shirts & Langarmshirts,True
1494058,663406178741,178741,3,2018-04-24,Piers Super Slim Jeans,4060586456604,Herren/Herrenmode/Jeans/Slim,TT_Denim,28.57,1,desktop,Safari,Macintosh,Germany,Bavaria,1004234,Herren,Jeans,True
444953,241240370688,370688,2,2018-03-05,Damen-T-Shirt mit Schrift-Print,4059491837165,Damen/Damenmode/T-Shirts & Langarmshirts/T-Shirts,TT_Denim,6.71,1,mobile,Chrome,Android,Germany,Saxony,1004916,Damen,T-Shirts & Langarmshirts,True
1099674,124928178199,178199,1,2018-07-22,gemustertes Blusen-Shirt,4060586068050,Damen/Damenmode/Blusen/kurzarm,TT_Casual,18.90,1,mobile,Safari,iOS,Germany,North Rhine-Westphalia,1004607,Damen,T-Shirts & Langarmshirts,True


In [51]:
camp_false = df[df.campaign_involved == False]

In [52]:
len(camp_false)

738453

In [53]:
camp_false[camp_false["article_id"]==4011694579266].head()

,visit_id,user_id,visit_number,date,article_name,article_id,category,brand,price,quantity,device_category,device_browser,device_os,country,region,city_id,product_line,product_group,campaign_involved
35271,505889355464,355464,5,2017-11-13,Hip-Pants im 3er-Pack,4011694579266,Herren/Herrenmode/Underwear & Pyjamas,TT_Casual,15.12,1,mobile,Firefox,Android,Germany,Berlin,1003854,Herren,Underwear & Pyjamas,False
35274,383063313091,313091,2,2018-03-24,Hip-Pants im 3er-Pack,4011694579266,Herren/Herrenmode/Underwear & Pyjamas,TT_Casual,18.90,1,tablet,Safari,iOS,Germany,Schleswig-Holstein,9048708,Herren,Underwear & Pyjamas,False
35277,201946434776,434776,6,2018-07-29,Hip-Pants im 3er-Pack,4011694579266,Herren/Herrenmode/Underwear & Pyjamas,TT_Casual,25.20,1,desktop,Firefox,Windows,Germany,Schleswig-Holstein,1004866,Herren,Underwear & Pyjamas,False
35281,333769606504,606504,1,2018-09-17,Hip-Pants im 3er-Pack,4011694579266,Herren/Herrenmode/Underwear & Pyjamas,TT_Casual,20.16,1,desktop,Chrome,Macintosh,Germany,Berlin,1003854,Herren,Underwear & Pyjamas,False
35295,14858564131,564131,1,2019-02-04,Hip-Pants im 3er-Pack,4011694579266,Herren/Herrenmode/Underwear & Pyjamas,TT_Casual,16.80,1,desktop,Firefox,Macintosh,Germany,Lower Saxony,1004536,Herren,Underwear & Pyjamas,False


In [54]:
camp_true[camp_true["article_id"]==4011694579266].head()

,visit_id,user_id,visit_number,date,article_name,article_id,category,brand,price,quantity,device_category,device_browser,device_os,country,region,city_id,product_line,product_group,campaign_involved
35270,723099348206,348206,6,2017-11-13,Hip-Pants im 3er-Pack,4011694579266,Herren/Herrenmode/Underwear & Pyjamas,TT_Casual,21.42,1,mobile,Samsung Internet,Android,Germany,Berlin,1003854,Herren,Underwear & Pyjamas,True
35272,539377527591,527591,6,2018-08-29,Hip-Pants im 3er-Pack,4011694579266,Herren/Herrenmode/Underwear & Pyjamas,TT_Casual,25.21,1,mobile,Samsung Internet,Android,Germany,Bavaria,1004269,Herren,Underwear & Pyjamas,True
35273,54340111696,111696,2,2018-09-08,Hip-Pants im 3er-Pack,4011694579266,Herren/Herrenmode/Underwear & Pyjamas,TT_Casual,18.90,1,desktop,Chrome,Chrome OS,Germany,Lower Saxony,9048874,Herren,Underwear & Pyjamas,True
35275,492320350403,350403,6,2018-10-26,Hip-Pants im 3er-Pack,4011694579266,Herren/Herrenmode/Underwear & Pyjamas,TT_Casual,14.70,2,desktop,Chrome,Windows,Germany,North Rhine-Westphalia,1004761,Herren,Underwear & Pyjamas,True
35276,31699418198,418198,1,2018-09-13,Hip-Pants im 3er-Pack,4011694579266,Herren/Herrenmode/Underwear & Pyjamas,TT_Casual,18.90,1,mobile,Chrome,Android,Germany,Thuringia,1004999,Herren,Underwear & Pyjamas,True


Certainly campaign_involved can not mean discounts and it is still unclear what it does.

## Number of occurence of countries

In [55]:
df.country.value_counts().head()

Germany        2756836
Austria         196950
Netherlands      22547
Switzerland       5425
Belgium           2434
Name: country, dtype: int64

## Check for missing dates 

In [56]:
from datetime import date, timedelta
date_set = set(df.date.min() + timedelta(x) for x in range((df.date.max() - df.date.min()).days))
missing = sorted(date_set - set(df.date))
missing

[]

As we can see there are no missing days in the date Data.

<a id='5'></a>

# 5. Feature Engineering

<a id='5.1'></a>

## 5.1 Cities by Code

In [57]:
df_geotargets.head()

,Criteria ID,Name,Canonical Name,Parent ID,Country Code,Target Type,Status
0,1000002,Kabul,"Kabul,Kabul,Afghanistan",9075393.0,AF,City,Active
1,1000003,Luanda,"Luanda,Luanda Province,Angola",9070431.0,AO,City,Active
2,1000004,The Valley,"The Valley,Anguilla",2660.0,AI,City,Active
3,1000010,Abu Dhabi,"Abu Dhabi,Abu Dhabi,United Arab Emirates",9041082.0,AE,City,Active
4,1000011,Ajman,"Ajman,Ajman,United Arab Emirates",9047096.0,AE,City,Active


In [58]:
#Criteria ID as index
df_geotargets_clean = df_geotargets.set_index("Criteria ID")

In [59]:
#take just the Name column
df_geotargets_clean = df_geotargets_clean["Name"]

In [60]:
#get a dictionary from the dataframe
geodic = df_geotargets_clean.to_dict()

In [61]:
#entry for not set ids
geodic[0] = "(not set)"

In [62]:
#replace not set with a number for a moment to change to int
df["city_id"] = df["city_id"].replace({"(not set)": 0})

In [63]:
#change city_id type to int
df = df.astype({'city_id': 'int64'})

In [64]:
#map cities to dataframe with new column city
df["city"] = df["city_id"].map(geodic)

In [65]:
#nans in cities by ids that were not found
df.isna().sum()

visit_id               0
user_id                0
visit_number           0
date                   0
article_name           0
article_id             0
category               0
brand                  0
price                  0
quantity               0
device_category        0
device_browser         0
device_os              0
country                0
region                 0
city_id                0
product_line           0
product_group          0
campaign_involved      0
city                 640
dtype: int64

In [66]:
#df[df["city"].isnull()]

In [67]:
#fill the nans with (not set)
df.fillna(value={"city": "(not set)"}, inplace=True)

In [68]:
#df[df["city"].isnull()]["country"].value_counts()

In [69]:
df_geotargets["Target Type"].value_counts()

Postal Code               48391
City                      38097
Neighborhood               4631
County                     3417
Municipality               2124
Province                   1131
District                    948
Region                      909
Congressional District      441
Airport                     395
Department                  240
State                       235
University                  219
Country                     213
City Region                 183
Governorate                 121
National Park                96
Borough                      88
Prefecture                   49
Okrug                        28
Canton                       26
Autonomous Community         22
TV Region                    14
Union Territory               7
Territory                     4
Name: Target Type, dtype: int64

<a id='5.2'></a>

## 5.2 Timespan

In [71]:
year_list = pd.DatetimeIndex(df['date']).year  

In [72]:
year_list.unique()

Int64Index([2018, 2017, 2019], dtype='int64', name='date')

In [73]:
df.date.max()

Timestamp('2019-02-28 00:00:00')

In [74]:
df.date.min()

Timestamp('2017-11-01 00:00:00')

We have data from 2017-11-01 till 2019-02-28.

<a id='5.3'></a>

## 5.3 Weekdays
monday 0 , ... , sunday 6

In [75]:
df["day_of_week"] = pd.to_datetime(df["date"]).dt.dayofweek

<a id='5.4'></a>

## 5.4 Article Genre

I will create new features for a slim version of the categories. Hence only women, men, kids and home & living.

In [76]:
#put all features describing the article type together for easier use
df["genre_column_placeholder"] = df["article_name"] + "/" + df["category"] + "/" + df["product_line"] + "/" + df["product_group"]

In [77]:
df.genre_column_placeholder = df.genre_column_placeholder.astype("str")

In [78]:
#new column for articles of women genre
def check_women(item):
    return "" if re.search(r"([Dd]{1}ame[ns]{1})|([Ww]{1}om[ea]{1}n)", item) == None else "Women"

df['women_article'] = df['genre_column_placeholder'].apply(check_women)


In [79]:
#new column for articles of men genre
def check_men(item):
    return "" if re.search(r"([Hh]{1}e[r]{1,2}en)|(Men\b)|(\b[Mm]en\b)", item) == None else "Men"

df['men_article'] = df['genre_column_placeholder'].apply(check_men)


In [80]:
#new column for articles of kids/teenager/babies genre
def check_kids(item):
    return "" if re.search(r"([Kk]{1}inder)|(\b[Kk]{1}id[s]{0,1})|([Jj]{1}ungen)|([Mm]{1}ädchen)|([Tt]{1}een[s]{0,1})|([Bb]{1}aby)|([Bb]{1}abies)", item) == None else "Kids"

df['kids_article'] = df['genre_column_placeholder'].apply(check_kids)


In [81]:
#new column for articles of home&living genre
def check_hl(item):
    return "" if re.search(r"([Hh]{1}ome)|([Ll]iving)", item) == None else "Home&living"

df['homeandliving_article'] = df['genre_column_placeholder'].apply(check_hl)


In [82]:
df = df.assign(category_slim = df.women_article + df.men_article + df.kids_article + df.homeandliving_article)

In [83]:
def putslash(item): 
  
    # regex [A-Z][a-z]* means any string starting  
    # with capital character followed by many  
    # lowercase letters  
    words = re.findall('[A-Z][a-z]*', item) 
  
    # Change first letter of each word into lower 
    # case 
    result = [] 
    for word in words: 
        word = chr( ord (word[0]) + 32) + word[1:] 
        result.append(word) 
    return '/'.join(result)

In [84]:
df["category_slim"] = df["category_slim"].apply(putslash)

In [85]:
df["category_slim"] = df["category_slim"].replace("", "(not set)")

In [86]:
df.women_article = df.women_article.replace({"": 0, "Women": 1})
df.men_article = df.men_article.replace({"": 0, "Men": 1})
df.kids_article = df.kids_article.replace({"": 0, "Kids": 1})
df.homeandliving_article = df.homeandliving_article.replace({"": 0, "Home&living": 1})

In [87]:
article_diff = df[(df.women_article + df.men_article + df.kids_article + df.homeandliving_article) == 0]

In [88]:
article_diff.product_group.value_counts()

Schuhe & Accessoires        7885
(not set)                   7284
Herbst Outfits              6384
T-Shirts & Langarmshirts    1052
Neu im Sale                  427
Festival Styles              326
SAFARI STYLES                212
Summer Sale Highlights       121
Bold Summer Colors            83
Black & White                 46
Alles zum halben Preis        24
T-Shirts unter 20 €            9
Partyoutfits                   3
Revolverheld Kollektion        2
Mexico Styles                  2
Hosen                          1
Name: product_group, dtype: int64

In [89]:
article_diff.category.value_counts().head()

(not set)         7000
Herbst Outfits    4431
Athleisure        3881
Schuhe            1812
Winterpullover     818
Name: category, dtype: int64

In [90]:
len(article_diff)

23861

So we are left with only 23861 articles without any of the new article groups.

In [91]:
#number of articles that got multiple article groups
mult_article = df[df.women_article + df.men_article + df.kids_article + df.homeandliving_article > 1]

In [92]:
mult_article.sample(n=5)

,visit_id,user_id,visit_number,date,article_name,article_id,category,brand,price,quantity,device_category,device_browser,device_os,country,region,city_id,product_line,product_group,campaign_involved,city,day_of_week,genre_column_placeholder,women_article,men_article,kids_article,homeandliving_article,category_slim
1833340,381712336179,336179,36,2018-05-04,Nela Extra Skinny Ankle Hose,4060868042549,Damen/Damenmode/Hosen,(not set),33.61,1,mobile,Chrome,Android,Germany,Bavaria,9048196,Herren,Hosen,True,Bad Windsheim,4,Nela Extra Skinny Ankle Hose/Damen/Damenmode/H...,1,1,0,0,women/men
1962522,469102480815,480815,15,2019-01-05,Wollmantel mit Reverskragen,4060868232018,"Herren/Herrenmode/Jacken, Mäntel & Westen/Mänt...",TT_Denim,56.71,1,mobile,Samsung Internet,Android,Germany,Bavaria,1004234,Damen,Herren,True,Munich,5,Wollmantel mit Reverskragen/Herren/Herrenmode/...,1,1,0,0,women/men
2020850,181999415616,415616,5,2018-11-25,Kleid mit Taillenbund,4060868301332,Damen,TT_Casual,31.29,1,mobile,Android Webview,Android,Germany,Mecklenburg-Vorpommern,1004446,Herren,Business,True,Greifswald,6,Kleid mit Taillenbund/Damen/Herren/Business,1,1,0,0,women/men
1988920,816228219635,219635,28,2018-08-31,Steppjacke,4060868276852,"Damen/Damenmode/Jacken, Mäntel & Westen/Jacken",TT_Denim,58.82,1,mobile,Samsung Internet,Android,Germany,Bavaria,1004200,Herren,Damen,True,Ingolstadt,4,"Steppjacke/Damen/Damenmode/Jacken, Mäntel & We...",1,1,0,0,women/men
134686,735512239062,239062,14,2018-06-05,Herren-Super-Slim-Jeans – Piers,4057655717667,Feeling Blue Men,TT_Denim,31.50,1,mobile,Safari,iOS,Germany,North Rhine-Westphalia,1004707,Damen,Herren,True,Munster,1,Herren-Super-Slim-Jeans – Piers/Feeling Blue M...,1,1,0,0,women/men


In [93]:
len(mult_article)

65808

Having 65808 articles with multiple article groups. Most of them do make at least some sense, like a jacket for a teenager also having the product_line Men.

<a id='5.5'></a>

## 5.5 Total price per article purchase

In [94]:
df["total_price"] = df["price"] * df["quantity"]

## Ordering columns

In [95]:
#code for checking the index of a certain column
df.columns.get_loc("price")

8

In [96]:
#check name of certain index
df.columns[27]

'total_price'

In [97]:
#reordering the columns
cols = df.columns.tolist()
cols = cols[0:4] + [cols[20]] + cols[4:6] + cols[8:10] + [cols[27]] \
        + cols[13:15] + [cols[19]] + [cols[15]] + cols[22:26] + [cols[18]] \
        + [cols[6]] + [cols[26]] + cols[7:8] + cols[16:18] + [cols[21]] + cols[10:13]
print("Check if any column was skipped when reordering (True: all columns fetched):", len(cols) == len(df.columns))
df = df.reindex(columns = cols)

Check if any column was skipped when reordering (True: all columns fetched): True


<a id='5.6'></a>

## 5.6 Value and number of items of shopping carts

In [98]:
shopcart = df.groupby(["visit_id"]).sum()

In [99]:
#take just the Name column
df_shopcartvalue_clean = shopcart["total_price"]
df_shopcartnumber_clean = shopcart["quantity"]

In [100]:
#get a dictionary from the dataframe
cartdicval = df_shopcartvalue_clean.to_dict()
cartdicnum = df_shopcartnumber_clean.to_dict()
cartdiffart = df.visit_id.value_counts().to_dict()

In [101]:
df["shop_cart_value"] = df["visit_id"].map(cartdicval)
df["shop_cart_item_count"] = df["visit_id"].map(cartdicnum)
df["shop_cart_unique_articles"] = df["visit_id"].map(cartdiffart)

In [102]:
#reordering the columns
cols = df.columns.tolist()
cols = cols[0:10] + cols[28:31] + cols[10:28]
print("Check if any column was skipped when reordering (True: all columns fetched):", len(cols) == len(df.columns))
df = df.reindex(columns = cols)

Check if any column was skipped when reordering (True: all columns fetched): True


<a id='5.7'></a>

## 5.7 Stockprice per day

In [103]:
df_stock["timestamp"]= pd.to_datetime(df_stock["timestamp"],format='%Y-%m-%d')

In [104]:
#fill missing days ( weekends )
df_stock = df_stock.resample('D',on="timestamp").mean()

In [105]:
#fill the nans of the missings above with the average of their direct set neighbours
df_stock = (df_stock.ffill() + df_stock.bfill())/2

In [106]:
df_stock.head()

,open,high,low,close,volume
timestamp,,,,,
2010-03-26,11.9637,12.04200,11.4576,11.8257,2767746.0
2010-03-27,11.8740,11.97755,11.5404,11.8349,1489778.5
2010-03-28,11.8740,11.97755,11.5404,11.8349,1489778.5
2010-03-29,11.7843,11.91310,11.6232,11.8441,211811.0
2010-03-30,11.8073,11.91770,11.6600,11.8717,80096.0


In [107]:
#take just the close column
df_stock_clean = df_stock["close"]

In [108]:
#get a dictionary from the dataframe
stockdic = df_stock_clean.to_dict()

In [109]:
#map stock to dataframe with new column stock_value
df["stock_value"] = df["date"].map(stockdic)

In [110]:
#check for nans that were not set 
df.isna().sum()

visit_id                     0
user_id                      0
visit_number                 0
date                         0
day_of_week                  0
article_name                 0
article_id                   0
price                        0
quantity                     0
total_price                  0
shop_cart_value              0
shop_cart_item_count         0
shop_cart_unique_articles    0
country                      0
region                       0
city                         0
city_id                      0
women_article                0
men_article                  0
kids_article                 0
homeandliving_article        0
campaign_involved            0
category                     0
category_slim                0
brand                        0
product_line                 0
product_group                0
genre_column_placeholder     0
device_category              0
device_browser               0
device_os                    0
stock_value                  0
dtype: i

In [111]:
#easy dataframe to check the stockprice development 
df.drop_duplicates("date").sort_values("date").head(5)

,visit_id,user_id,visit_number,date,day_of_week,article_name,article_id,price,quantity,total_price,shop_cart_value,shop_cart_item_count,shop_cart_unique_articles,country,region,city,city_id,women_article,men_article,kids_article,homeandliving_article,campaign_involved,category,category_slim,brand,product_line,product_group,genre_column_placeholder,device_category,device_browser,device_os,stock_value
2325,14930374362,374362,3,2017-11-01,2,Tennis-Socken im Dreierpack,4002702234727,5.01,1,5.01,32.69,5,5,Germany,North Rhine-Westphalia,Neuss,1004708,0,1,0,0,True,Herren/Herrenschuhe & Accessoires/Accessoires/...,men,TT_Casual,Herren,Schuhe & Accessoires,Tennis-Socken im Dreierpack/Herren/Herrenschuh...,mobile,Chrome,Android,8.859
5836,645245525600,525600,2,2017-11-02,3,Basic Socken im Doppelpack,4002702235090,5.84,2,11.68,108.55,5,4,Germany,Baden-Wurttemberg,(not set),0,0,1,0,0,True,Herren/Herrenschuhe & Accessoires/Accessoires/...,men,TT_Casual,Herren,Schuhe & Accessoires,Basic Socken im Doppelpack/Herren/Herrenschuhe...,desktop,Firefox,Windows,9.131
2849,448437326618,326618,1,2017-11-03,4,Tennis-Socken im Dreierpack,4002702234765,5.52,2,11.04,83.18,7,6,Germany,North Rhine-Westphalia,Wermelskirchen,1004761,0,1,0,0,False,Herren/Herrenschuhe & Accessoires/Accessoires/...,men,TT_Casual,Herren,Schuhe & Accessoires,Tennis-Socken im Dreierpack/Herren/Herrenschuh...,desktop,Firefox,Windows,9.100
2512,813967575820,575820,11,2017-11-04,5,Tennis-Socken im Dreierpack,4002702234758,6.68,2,13.36,78.38,4,3,Germany,Baden-Wurttemberg,Esslingen,1003938,0,1,0,0,True,Herren/Herrenschuhe & Accessoires/Accessoires/...,men,TT_Casual,Herren,Schuhe & Accessoires,Tennis-Socken im Dreierpack/Herren/Herrenschuh...,mobile,Samsung Internet,Android,9.090
455,754081156504,156504,1,2017-11-05,6,Geschenkgutschein 50 EUR,200000000050,50.00,1,50.00,50.00,1,1,Germany,Schleswig-Holstein,Flensburg,1004855,0,1,0,0,False,Herren,men,(not set),Herren,Herren,Geschenkgutschein 50 EUR/Herren/Herren/Herren,desktop,Firefox,Windows,9.090


<a id='5.8'></a>

## 5.8 First date of purchase and purchase type

In [112]:
#help feature with a edited date
df['year_month'] = df['date'].map(lambda date: 100*date.year + date.month)

In [113]:
#finding the date where users bought for the first time in our timeline
min_purchase = df.groupby("user_id").year_month.min().reset_index()
min_purchase.columns = ["user_id","min_purchase_date"]

#merge
df = pd.merge(df, min_purchase, on="user_id")

In [114]:
#create a feature user_type with first or established depending on time of purchase
df["user_type"] = "first"
df.loc[df["year_month"]>df["min_purchase_date"], "user_type"] = "established"

<a id='5.9'></a>

## 5.9 Final Feature dictionary

| Feature No. | Feature Name | Description |
| :-: | :-: | :-: |
| 1 | visit_id | Unique ID consisting of the ID of a single shopping cart and the ID of the user |
| 2 | user_id | Unique ID of a user |
| 3 | visit_number | Not entirely clear; either number of visits to the shop or number of total purchases |
| 4 | date | Date the purchase was made |
| 5 | day_of_week | Day when the purchase was made; 0 for monday till 6 for sunday |
| 6 | article_name | Name of the article that was bought |
| 7 | article_id | ID of the article that was bought |
| 8 | price | Price of the article |
| 9 | quantity | Number of the article that were bought |
| 10 | total_price | Total price of the article purchase consisting of price * quantity |
| 11 | shop_cart_value | Value of the whole shopping cart of a single visit_id |
| 12 | shop_cart_item_count | Number of items in the shopping cart of a single visit_id |
| 13 | shop_cart_unique_articles | Number of unique items in the shopping cart of a single visit_id |
| 14 | country | country location of the user |
| 15 | region | region location of the user |
| 16 | city | city location of the user |
| 17 | city_id | geoID of the town location of the user |
| 18 | women_article | women category - 1 yes , 0 no |
| 19 | men_article | men category - 1 yes , 0 no |
| 20 | kids_article | kids category - 1 yes , 0 no |
| 21 | homeandliving_article | home & living category - 1 yes , 0 no |
| 22 | campaign_involved | still not completely clear but after some research it might stand for the cases where users came to the shop via advertisement. When you get to the shop this way the url contains a part for campaign: for example "utm_campaign=de_retargeting_criteo"|
| 23 | category | Category the article falls in ( partly the path in the webshop to the article ) |
| 24 | category_slim | featured category with women, men, kids and home&living |
| 25 | brand | Brand of the article |
| 26 | product_line | Product line of the article |
| 27 | product_group | Product group of the article |
| 28 | genre_column_placeholder | Sum of article name, category and product lines/groups to better work with |
| 29 | device_category | Category the user device falls in |
| 30 | device_browser | Category the user browser falls in |
| 31 | device_os | Category the user OS falls in |
| 32 | stock_value | company stock price at this day |
| 33 | year_month | feature with year and month as int |
| 34 | min_purchase_date | date of the first purchase in our timeline of this user |
| 35 | user_type | type of the purchase, first or established |

## save dataframe to pickle

In [115]:
df.to_pickle('Data/dataframefull.pickle')

# Future Work
- There are still features that have no clear meaning. So find those out.
- Get the data for the anonymous users
- Get more data for the articles ( like color, sizes, revenue for the price sold etc.)